# Proyecto ETL
#### Grupo 1:  Gledson - Eva - Sara - Rubén - Luis Q

### - - Importación de módulos y librerías - -

In [3]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import os
import locale

from datetime import datetime
from pprint import pprint

#  Se establece la configuración regional
configuracion_local = locale.setlocale(locale.LC_ALL, ("es_ES", "UTF-8"))

print (f"Se establece la configuración regional a {configuracion_local}")

Se establece la configuración regional a es_ES.UTF-8


# Extracción de datos

Se definen funciónes para la extracción de datos

In [4]:
def extrae_datos_meneos (lista_meneos, año_busqueda):
    datos = []
    continuar = True
    for meneo in lista_meneos:
        dicc_aux = {}
        
        # Se obtiene en primer lugar el año
        try:
            fecha_pub = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")[1].text.split(" ")[0]
            año = datetime.strptime(fecha_pub,"%d/%m/%Y").year if len(fecha_pub) > 5 else 2024
        except:
            año = False
        
        # Si el año detectado es igual que el año buscado, se continúa con la importación
        if año == año_busqueda:
            ###################################    Titular    ####################################################################
            try:
                dicc_aux["Titular"] = meneo.find("h2").text.replace("\n", " ").strip()
            except:
                dicc_aux["Titular"] = np.nan

            ###################################    Entradilla    ###################################################################
            try:
                dicc_aux["Entradilla"] = meneo.find("div", class_="news-content").text.replace("\n", " ").strip()
            except:
                dicc_aux["Entradilla"] = np.nan
                
            ###################################   Comunidad (Temática) ############################################################
            try:
                dicc_aux["Comunidad"] = meneo.find("a", class_="subname").text.strip()
            except:
                dicc_aux["Comunidad"] = np.nan

            ###################################    Usuario  ######################################################################
            try:
                dicc_aux["Usuario"] = meneo.find("div", class_="news-submitted").find("img")["alt"].strip()
            except:
                dicc_aux["Usuario"] = np.nan

            ###################################    Medio    ######################################################################
            try:
                dicc_aux["Medio"] = meneo.find("span", class_="showmytitle").text.strip()
            except:            
                dicc_aux["Medio"] = "meneame.net"

            ###################################    URL    ######################################################################
            try:
                dicc_aux["URL"] = "https://www.meneame.net" + meneo.find("a", class_="comments")["href"].strip()
            except:            
                dicc_aux["URL"] = np.nan

            ###################################    Fechas    ######################################################################    
            try:
                fechas = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")
                dicc_aux["Enviado"] = fechas[0].text.strip()
                dicc_aux["Publicado"] = fechas[1].text.strip()
            except:
                dicc_aux["Enviado"] = np.nan
                dicc_aux["Publicado"] = np.nan

            ###################################    Meneos    ######################################################################
            try:
                dicc_aux["Meneos"] = meneo.find("div", class_="votes").find("a").text.strip()
            except:
                dicc_aux["Meneos"] = np.nan

            ###################################    Click    ######################################################################
            try:
                dicc_aux["Clicks"] = meneo.find("div", class_="clics").find("span").text.strip()
            except:
                dicc_aux["Clicks"] = np.nan

            ###################################    Positivos    ###################################################################
            try:
                dicc_aux["Positivos"] = meneo.find("span", class_="positive-vote-number").text.strip()
            except:
                dicc_aux["Positivos"] = np.nan

            ###################################    Votos Anónimos    ##############################################################
            try:
                dicc_aux["Anonimos"] = meneo.find("span", class_="anonymous-vote-number").text.strip()
            except:
                dicc_aux["Anonimos"] = np.nan

            ###################################    Votos Negativos    ##############################################################
            try:
                dicc_aux["Negativos"] = meneo.find("span", class_="negative-vote-number").text.strip()
            except:
                dicc_aux["Negativos"] = np.nan

            ###################################    Número comentarios    ###########################################################
            try:
                dicc_aux["Comentarios"] = meneo.find("a", class_="comments")["data-comments-number"].strip()
            except:
                dicc_aux["Comentarios"] = np.nan

            ###################################    Karma    ########################################################################
            try:
                dicc_aux["Karma"] = meneo.find("span", class_="karma-number").text.strip()
            except:
                dicc_aux["Karma"] = np.nan

            datos.append(dicc_aux)

        # Si se ha terminado el año buscado se detendrá la importación
        elif año < año_busqueda:
            continuar = False

    return [datos, continuar]

def busca_pagina_inicio (año_elegido, total_paginas):
    num_pagina_año = {"2005": [10906,10878], "2006" : [10878,10442], "2007" : [10442,9759], "2008" : [9759,8981],
                    "2009" : [8981,8196], "2010" : [8196,7499], "2011" : [7499,6782], "2012" : [6782,6149],
                    "2013" : [6149,5623], "2014" : [5623,5101], "2015" : [5101,4609], "2016" : [4609,4214],
                    "2017" : [4214,3774], "2018" : [3774,3230], "2019" : [3230,2733], "2020" : [2733,2058],
                    "2021" : [2058,1409], "2022" : [1409,740], "2023" : [740,139]}

    lista_a = []
    for i in num_pagina_año.keys():
        lista_a.append([i,num_pagina_año[i][0] - num_pagina_año[i][1]])

    df_pags_año = pd.DataFrame(columns=["año", "cantidad"], data=lista_a)
    df_pags_año.set_index("año", inplace=True)
    df_pags_año["acumulado"] = df_pags_año["cantidad"].cumsum()
    df_pags_año["pag_inicio"] = total_paginas - df_pags_año["acumulado"]

    return df_pags_año.loc[str(año_elegido),"pag_inicio"]-1

#### Bucle para el Web Scrapping  ####

In [5]:
URL_BASE = "https://old.meneame.net/"

# pagina = 0
lista_aux = []
contador_paginas = 0
continuar_ejecucion = True

try:
    año = int(input("Introducir un año entre 2006 y 2023"))
    if año not in range(2006,datetime.now().year):
        año = datetime.now().year-1
        print("-"*73)      
        print(f" Año fuera de rango, se toma por defecto el último año finalizado ({año})")
        print("-"*73)
    else:
        print("-"*34)
        print(f" Extracción de datos del año {año}")
        print("-"*34)
except:
    año = datetime.now().year-1
    print("-"*69)
    print(f" Año incorrecto, se toma por defecto el último año finalizado ({año})")
    print("-"*69)

browser = webdriver.Chrome()
browser.get(f"{URL_BASE}")
total_paginas = int(BeautifulSoup(browser.page_source, "html.parser").find("div", class_="pages").find_all("a")[-2].text)
pagina = busca_pagina_inicio(año, total_paginas)

contador_para_pruebas = 0

try:
    while contador_para_pruebas < 10 and continuar_ejecucion == True:
        try:
            # Se solicita al navegador la apertura de la URL de la página
            browser.get(f"{URL_BASE}?page={pagina}")

            # Se obtiene el BeautifulSoup a partir del código fuente de la página
            Soup_pagina_meneos = BeautifulSoup(browser.page_source, "html.parser")

            # Se extrae el listado de noticias
            lista_meneos = Soup_pagina_meneos.find_all("div", class_="news-summary")

            # Se analiza el listado con la función de extracción
            resultado = extrae_datos_meneos(lista_meneos, año)
            
            # Se añaden los resultados de la extracción a una lista
            lista_aux.extend(resultado[0])

            # Se almacena el dato para la parada del bucle
            continuar_ejecucion = resultado[1]

            # Se aumenta el contador de páginas
            pagina += 1
            contador_paginas += 1

            contador_para_pruebas += 1
        except:
            print(f"Error en página {pagina}")
except:
    print("Ocurrido un error inesperado... ¿Estaba cargada la función?")

print(f" Total paginas procesadas: {contador_paginas}")
print(f" Total noticias importadas: {len(lista_aux)}")

browser.close()

----------------------------------
 Extracción de datos del año 2009
----------------------------------
Total paginas procesadas: 10
Total noticias importadas: 206


#### Creación del DataFrame con la lista de datos extraida

In [6]:
df_municipios = pd.read_csv("Data/Municipios.csv", sep = ";")
df = pd.DataFrame(lista_aux)
df.head(1)

,Titular,Entradilla,Comunidad,Usuario,Medio,URL,Enviado,Publicado,Meneos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma
0,Intro de Star Wars al estilo El Equipo A,¿Qué hubiera sucedido si en la versión remaste...,mnm,Bonzaitrax,abadiadigital.com,https://www.meneame.net/story/intro-star-wars-...,31/12/2009 21:11,31/12/2009 23:25,352,NaN,158,194,8,20,476
1,Australia: científicos desarrollan un sistema ...,Los grandes tiburones blancos ahora serán loca...,mnm,Tanatos,ecologiablog.com,https://www.meneame.net/story/australia-cienti...,31/12/2009 18:13,31/12/2009 22:10,187,NaN,107,80,0,30,471
2,Donación multiorgánica en hospital de Cartagen...,Una donación multiorgánica de un paciente del ...,mnm,Tanatos,diariosur.es,https://www.meneame.net/story/donacion-multior...,31/12/2009 19:29,31/12/2009 20:55,401,NaN,177,224,0,30,489
3,El 85% de los periodistas no rebisa lo que esc...,Según un estudio de la Universidad de Strand C...,mnm,painful,elmundotoday.com,https://www.meneame.net/story/85-periodistas-n...,31/12/2009 10:00,31/12/2009 19:50,370,NaN,169,201,4,50,548
4,El repunte del porno en Navidad,"La Navidad, esa época del año familiar, de reg...",mnm,hovercraft85,ensilicio.com,https://www.meneame.net/story/repunte-del-porn...,31/12/2009 16:40,31/12/2009 19:15,410,NaN,203,207,0,40,640


# TRANSFORMACIÓN

In [139]:
def franja_horaria (fecha):
    hora = fecha.hour
    if hora in range(6, 12):
        return "Mañana"
    elif hora in range(12, 16):
        return "Mediodía"
    elif hora in range(16, 21):
        return "Tarde"
    elif hora in range(21, 24):
        return "Noche"
    elif hora in range(0, 6):
        return "Madrugada"
    
lista_municipios = tuple(df_municipios["Nombre1"])
lista_municipios2 = tuple(df_municipios["Nombre2"])

def funcion_municipios(x):    
    municipio_encontrado = "No encontrado"
    for municipio1, municipio2 in zip(lista_municipios,lista_municipios2):
        if municipio1 in x:            
            municipio_encontrado = municipio1
            break
        elif municipio2 in x:
            municipio_encontrado = municipio1
            break
    
    return municipio_encontrado

In [140]:
df['Enviado'] = pd.to_datetime(df['Enviado'], format="%d/%m/%Y %H:%M")
df['Publicado'] = pd.to_datetime(df['Publicado'], format="%d/%m/%Y %H:%M")
df[["Meneos", "Clicks", "Positivos", "Anonimos","Negativos", "Comentarios", "Karma"]] = df[["Meneos", "Clicks", "Positivos", "Anonimos","Negativos", "Comentarios", "Karma"]].astype("int64")

In [141]:
df['Delay'] = df['Publicado'] - df['Enviado']
df["Franja Horaria"] = df["Publicado"].apply(franja_horaria)
df["Día de la Semana"] = df["Publicado"].apply(lambda x : x.strftime("%A").capitalize())
df["Mes"] = df["Publicado"].apply(lambda x : x.strftime("%B").capitalize())

trimestres = {1: "1er Trimestre", 2: "2do Trimestre", 3: "3er Trimestre", 4: "4to Trimestre"}
df['Trimestre'] = df['Publicado'].dt.quarter.map(trimestres)
df["Municipio"] = df["Titular"].apply(funcion_municipios)

indices = df[df["Municipio"] == "No encontrado"].index.to_list()
for i in indices:
    df.at[i, "Municipio"] = funcion_municipios(df.loc[i, "Entradilla"])
    
df = df.merge(df_municipios[["Provincia","Longitud", "Latitud","Nombre1"]],
              left_on="Municipio", right_on="Nombre1",how="left").drop("Nombre1", axis=1)

df["Provincia"] = df["Provincia"].replace(np.nan, "No encontrado")
df["Longitud"] = df["Longitud"].str.replace(",", ".")
df["Latitud"] = df["Latitud"].str.replace(",", ".")
df[["Latitud", "Longitud"]] = df[["Latitud", "Longitud"]].astype("float64")
df["Longitud"] = df["Longitud"].replace(np.nan, 0)
df["Latitud"] = df["Latitud"].replace(np.nan, 0)

# CARGA

In [142]:
TOKEN =  os.getenv("AIRTABLE_TOKEN") # Usuario

BASE_ID = "appOnz4LtIdKAOYfo" # Base

airtable_base_url = "https://api.airtable.com/v0"

# Headers
headers = {"Authorization" : f"Bearer {TOKEN}",
           "Content-Type"  : "application/json"}

#  json para la creación de la tabla

In [143]:
nombres_columnas = {"name" : f"Meneame_{año}",
        "description" : f"Listado de meneos de la portada general de {año}",
        "fields": [
            {
                "name" : "Titular",
                "type" : "singleLineText"
            },
            {
                "name" : "Entradilla",
                "type" : "multilineText"
            },            
            {
                "name" : "Comunidad",
                "type" : "singleSelect",
                "options" : {
                    "choices":[{                        
                        "name" : "politica"
                    },
                    {                        
                        "name" : "ciencia"
                    }
                    ]
                }
            },
            {
                "name" : "Usuario",
                "type" : "singleLineText"
            },
            {
                "name" : "Medio",
                "type" : "singleLineText"
            },
            {
                "name" : "URL",
                "type" : "url"
            },
            {
                "name" : "Enviado",
                "type" : "dateTime",
                "options": {
                    "timeZone" : "Europe/Madrid",
                    "dateFormat" : {
                        "format" : "YYYY-MM-DD",
                        "name": "iso"
                    },
                    "timeFormat" : {
                        "format": "HH:mm",
                        "name": "24hour"
                    }
                }
            },
            {
                "name" : "Publicado",
                "type" : "dateTime",
                "options": {
                    "timeZone" : "Europe/Madrid",
                    "dateFormat" : {
                        "format" : "YYYY-MM-DD",
                        "name": "iso"
                    },
                    "timeFormat" : {
                        "format": "HH:mm",
                        "name": "24hour"
                    }
                }
            },            
            {
                "name" : "Meneos",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Clicks",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Positivos",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Anonimos",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Negativos",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Comentarios",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            
            {
                "name" : "Karma",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },            
            {
                "name" : "Delay",
                "type" : "duration",
                "options" : {
                    "durationFormat" : "h:mm"
                }
            },
            {
                "name" : "Franja Horaria",
                "type" : "singleLineText"
            },
            {
                "name" : "Día de la Semana",
                "type" : "singleLineText"
            },
            {
                "name" : "Mes",
                "type" : "singleLineText",
            },
            {
                "name" : "Trimestre",
                "type" : "singleLineText"
            },
            {
                "name" : "Municipio",
                "type" : "singleLineText"
            },
            {
                "name" : "Provincia",
                "type" : "singleLineText"
            },
            {
                "name" : "Longitud",
                "type" : "number",
                "options" : {
                    "precision" : 8
                }
            },
            {
                "name" : "Latitud",
                "type" : "number",
                "options" : {
                    "precision" : 8
                }
            }            
        ]
        }

In [144]:
endpoint_crea_tabla = f"https://api.airtable.com/v0/meta/bases/{BASE_ID}/tables"
    
response = requests.post(url = endpoint_crea_tabla, json= nombres_columnas, headers = headers)

print(f"endpoint: {response.url}")
print(f"response: {response.status_code}")
print("-"*120)
pprint(response.json(), sort_dicts = False)
print("-"*120)

TABLE_ID = response.json()["id"]

endpoint: https://api.airtable.com/v0/meta/bases/appOnz4LtIdKAOYfo/tables
response: 200
------------------------------------------------------------------------------------------------------------------------
{'id': 'tblS5eSikTVZzeujZ',
 'name': 'Meneame_2023',
 'description': 'Listado de meneos de la portada general de 2023',
 'primaryFieldId': 'fldAORlCEIHEqQ5Xt',
 'fields': [{'type': 'singleLineText',
             'id': 'fldAORlCEIHEqQ5Xt',
             'name': 'Titular'},
            {'type': 'multilineText',
             'id': 'fldryf0p80AmcBqa9',
             'name': 'Entradilla'},
            {'type': 'singleSelect',
             'options': {'choices': [{'id': 'selOr35gLQD2DU43J',
                                      'name': 'politica',
                                      'color': 'grayLight2'},
                                     {'id': 'sel47RUnvx8ltVe1l',
                                      'name': 'ciencia',
                                      'color': 'grayLight2'}]

In [145]:
df['Enviado'] = df['Enviado'].astype("str")
df['Publicado'] = df['Publicado'].astype("str")
df['Delay'] = df['Delay'].astype("str")

In [146]:
# Endpoint
endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

datos_json = [{"fields" : df.iloc[i, :].to_dict()} for i in range(df.shape[0])]

for i in range(0, df.shape[0], 10):
    data = {"records" : datos_json[i : i + 10],
            "typecast": True}
    
    response = requests.post(url = endpoint, json = data, headers = headers) # POST
    pprint (response.json())


{'records': [{'id': 'recHylzVPUy5XQNp2', 'createdTime': '2024-03-21T21:38:45.000Z', 'fields': {'Titular': 'Yanis Varoufakis: Suspended el Apartheid Israelí de los deportes internacionales [EN]', 'Entradilla': 'Como una vez suspendimos el Apartheid de Sudáfrica. El apartheid debe terminar. De una vez por todas. ¡Firma ya la petición de DiEM25! internal.diem25.org/en/petitions/93', 'Comunidad': 'politica', 'Usuario': 'dmeijide', 'Medio': 'twitter.com', 'URL': 'https://www.meneame.net/story/yanis-varoufakis-suspender-apartheid-israel-deportes', 'Enviado': '2023-12-31T11:05:00.000Z', 'Publicado': '2023-12-31T21:40:00.000Z', 'Meneos': 649, 'Clicks': 1238, 'Positivos': 276, 'Anonimos': 373, 'Negativos': 9, 'Comentarios': 57, 'Karma': 399, 'Delay': 2286000, 'Franja Horaria': 'Noche', 'Día de la Semana': 'Domingo', 'Mes': 'Diciembre', 'Trimestre': '4to Trimestre', 'Municipio': 'No encontrado', 'Provincia': 'No encontrado', 'Longitud': 0, 'Latitud': 0}}, {'id': 'recjwlHZS2ZDhar59', 'createdTime

In [148]:
params = {}

endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

datos = list()

while params.get("offset") != None or len(datos) == 0:
    
    response = requests.get(url = endpoint, headers = headers, params = params)
    
    print(f"response: {response.status_code}")
    print(f"endpoint: {response.url}")
    
    data = response.json()
    
    offset = data.get("offset")
    
    params["offset"] = offset
    
    datos.extend(data["records"])
    
print(len(datos))

# for indice, elem in enumerate(datos):
#     for llave in datos[indice]["fields"].keys():
#         print (datos[indice]["fields"].keys())

# df_descargado = pd.DataFrame(datos)
# df_descargado.head()

response: 200
endpoint: https://api.airtable.com/v0/appOnz4LtIdKAOYfo/tblS5eSikTVZzeujZ
response: 200
endpoint: https://api.airtable.com/v0/appOnz4LtIdKAOYfo/tblS5eSikTVZzeujZ?offset=itrX7QpwP8fIS57hs%2FrecRVXR5rs5QS569k
response: 200
endpoint: https://api.airtable.com/v0/appOnz4LtIdKAOYfo/tblS5eSikTVZzeujZ?offset=itrX7QpwP8fIS57hs%2FrecwI8lzL0XKVAgxQ
210
dict_keys(['Titular', 'Entradilla', 'Comunidad', 'Usuario', 'Medio', 'URL', 'Enviado', 'Publicado', 'Meneos', 'Clicks', 'Positivos', 'Anonimos', 'Negativos', 'Comentarios', 'Karma', 'Delay', 'Franja Horaria', 'Día de la Semana', 'Mes', 'Trimestre', 'Municipio', 'Provincia', 'Longitud', 'Latitud'])
dict_keys(['Titular', 'Entradilla', 'Comunidad', 'Usuario', 'Medio', 'URL', 'Enviado', 'Publicado', 'Meneos', 'Clicks', 'Positivos', 'Anonimos', 'Negativos', 'Comentarios', 'Karma', 'Delay', 'Franja Horaria', 'Día de la Semana', 'Mes', 'Trimestre', 'Municipio', 'Provincia', 'Longitud', 'Latitud'])
dict_keys(['Titular', 'Entradilla', 'Comunid

In [166]:
# GET Records
params = {"offset" : None}
df_airtable = pd.DataFrame()
while params.get("offset") != None or df_airtable.shape[0] == 0:
    response = requests.get(url = endpoint, headers = headers, params = params)
    print(response.url)
    print(f"response: {response.status_code}")
    params["offset"] = response.json().get("offset")
    # print(params.get("offset"))
    # pd.json_normalize(response.json()["records"][i]["fields"])]
    df_airtable = pd.concat([df_airtable, pd.json_normalize(response.json()["records"])], ignore_index = True)    
    pprint (response.json()["records"][1]["fields"])
    # sleep(0.5)

df_airtable.columns = [x.split(".")[1] if "." in x else x for x in df_airtable.columns]
df_airtable.drop(["id", "createdTime"], axis=1, inplace=True)
df_airtable

https://api.airtable.com/v0/appOnz4LtIdKAOYfo/tblS5eSikTVZzeujZ
response: 200
{'Anonimos': 231,
 'Clicks': 6385,
 'Comentarios': 145,
 'Comunidad': 'tecnología',
 'Delay': 1429200,
 'Día de la Semana': 'Domingo',
 'Entradilla': 'Las redes de telefonía están sufriendo una plaga de campañas '
               'de spam fraudulentas que aprovechan la falta de seguridad del '
               'protocolo con el que funciona la identificación de llamadas '
               'para engañar a las víctimas mostrando un número manipulado que '
               'pertenece a otro abonado. Así funciona el Caller ID spoofing.',
 'Enviado': '2023-12-31T02:13:00.000Z',
 'Franja Horaria': 'Mañana',
 'Karma': 419,
 'Latitud': 0,
 'Longitud': 0,
 'Medio': 'bandaancha.eu',
 'Meneos': 436,
 'Mes': 'Diciembre',
 'Municipio': 'No encontrado',
 'Negativos': 0,
 'Positivos': 205,
 'Provincia': 'No encontrado',
 'Publicado': '2023-12-31T08:50:00.000Z',
 'Titular': 'Cómo se manipula el identificador de llamadas para comete

,Titular,Entradilla,Comunidad,Usuario,Medio,URL,Enviado,Publicado,Meneos,Clicks,...,Karma,Delay,Franja Horaria,Día de la Semana,Mes,Trimestre,Municipio,Provincia,Longitud,Latitud
0,Guatemalteco podría ser condenado a cadena per...,Un policía murió de un infarto luego de forcej...,actualidad,peluchon,prensalibre.com,https://www.meneame.net/story/guatemalteco-pod...,2023-12-29T20:08:00.000Z,2023-12-29T23:40:00.000Z,386,3695,...,489,763200,Madrugada,Sábado,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
1,Cómo se manipula el identificador de llamadas ...,Las redes de telefonía están sufriendo una pla...,tecnología,provotector,bandaancha.eu,https://www.meneame.net/story/como-manipula-id...,2023-12-31T02:13:00.000Z,2023-12-31T08:50:00.000Z,436,6385,...,419,1429200,Mañana,Domingo,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
2,Pablo Iglesias pide el voto para el BNG y acon...,"El exlíder de Podemos apela a ""la inteligencia...",politica,Supercinexin,publico.es,https://www.meneame.net/story/pablo-iglesias-p...,2023-12-28T16:56:00.000Z,2023-12-28T20:40:00.000Z,345,1499,...,515,806400,Noche,Jueves,Diciembre,4to Trimestre,Iglesias,Burgos,-3.989414,42.298048
3,Sumar desvela un preacuerdo con Podemos y EU p...,Lo anunció el portavoz de la formación en la p...,actualidad,Beltenebros,eldiario.es,https://www.meneame.net/story/sumar-desvela-pr...,2023-12-27T18:07:00.000Z,2023-12-27T23:35:00.000Z,182,758,...,439,1180800,Madrugada,Jueves,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
4,Así deberían abrir los telediarios de las TVs ...,Así deberían abrir los telediarios de las TVs ...,actualidad,Delay,twitter.com,https://www.meneame.net/story/asi-deberian-abr...,2023-12-30T19:16:00.000Z,2023-12-30T21:55:00.000Z,647,2827,...,527,572400,Noche,Sábado,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,El Supremo tumba otra denuncia de Vox y critic...,La Sala de lo Penal confirma la inadmisión de ...,actualidad,onainigo,eldiario.es,https://www.meneame.net/story/supremo-tumba-ot...,2023-12-25T23:38:00.000Z,2023-12-26T10:45:00.000Z,559,847,...,528,2401200,Mañana,Martes,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
206,El eurodiputado Miguel Urbán nos explica lo qu...,El eurodiputado Miguel Urbán nos explica lo qu...,actualidad,MLeon,twitter.com,https://www.meneame.net/story/eurodiputado-mig...,2023-12-30T19:48:00.000Z,2023-12-30T20:10:00.000Z,668,4167,...,526,79200,Noche,Sábado,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
207,"""La mujer es un enemigo, el bebé es un enemigo...","Eliyahu Yossian, analista israelí y veterano d...",actualidad,--557077--,twitter.com,https://www.meneame.net/story/mujer-enemigo-be...,2023-12-27T11:57:00.000Z,2023-12-27T14:55:00.000Z,407,1302,...,430,640800,Mediodía,Miércoles,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
208,Gypsy Rose Blanchard sale de la cárcel tras cu...,"Gypsy Rose Blanchard, de 32 años, se declaró c...",actualidad,Cesc_,bbc.com,https://www.meneame.net/story/gypsy-rose-blanc...,2023-12-28T13:30:00.000Z,2023-12-28T18:15:00.000Z,85,2251,...,438,1026000,Tarde,Jueves,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000


In [153]:
df_airtable

,id,createdTime,fields.Titular,fields.Entradilla,fields.Comunidad,fields.Usuario,fields.Medio,fields.URL,fields.Enviado,fields.Publicado,...,fields.Karma,fields.Delay,fields.Franja Horaria,fields.Día de la Semana,fields.Mes,fields.Trimestre,fields.Municipio,fields.Provincia,fields.Longitud,fields.Latitud
0,rec0FQQglqO653OhC,2024-03-21T21:38:55.000Z,Guatemalteco podría ser condenado a cadena per...,Un policía murió de un infarto luego de forcej...,actualidad,peluchon,prensalibre.com,https://www.meneame.net/story/guatemalteco-pod...,2023-12-29T20:08:00.000Z,2023-12-29T23:40:00.000Z,...,489,763200,Madrugada,Sábado,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
1,rec0Ium6TDCPLOWtR,2024-03-21T21:38:48.000Z,Cómo se manipula el identificador de llamadas ...,Las redes de telefonía están sufriendo una pla...,tecnología,provotector,bandaancha.eu,https://www.meneame.net/story/como-manipula-id...,2023-12-31T02:13:00.000Z,2023-12-31T08:50:00.000Z,...,419,1429200,Mañana,Domingo,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
2,rec0dtk6ih8AMAAwO,2024-03-21T21:39:00.000Z,Pablo Iglesias pide el voto para el BNG y acon...,"El exlíder de Podemos apela a ""la inteligencia...",politica,Supercinexin,publico.es,https://www.meneame.net/story/pablo-iglesias-p...,2023-12-28T16:56:00.000Z,2023-12-28T20:40:00.000Z,...,515,806400,Noche,Jueves,Diciembre,4to Trimestre,Iglesias,Burgos,-3.989414,42.298048
3,rec11wS3NgeU2aH5X,2024-03-21T21:39:07.000Z,Sumar desvela un preacuerdo con Podemos y EU p...,Lo anunció el portavoz de la formación en la p...,actualidad,Beltenebros,eldiario.es,https://www.meneame.net/story/sumar-desvela-pr...,2023-12-27T18:07:00.000Z,2023-12-27T23:35:00.000Z,...,439,1180800,Madrugada,Jueves,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
4,rec1FACrS33QloI3D,2024-03-21T21:38:50.000Z,Así deberían abrir los telediarios de las TVs ...,Así deberían abrir los telediarios de las TVs ...,actualidad,Delay,twitter.com,https://www.meneame.net/story/asi-deberian-abr...,2023-12-30T19:16:00.000Z,2023-12-30T21:55:00.000Z,...,527,572400,Noche,Sábado,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,recyJD3ERSR305O20,2024-03-21T21:39:16.000Z,El Supremo tumba otra denuncia de Vox y critic...,La Sala de lo Penal confirma la inadmisión de ...,actualidad,onainigo,eldiario.es,https://www.meneame.net/story/supremo-tumba-ot...,2023-12-25T23:38:00.000Z,2023-12-26T10:45:00.000Z,...,528,2401200,Mañana,Martes,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
206,recyVPNch5zZ3tvOS,2024-03-21T21:38:50.000Z,El eurodiputado Miguel Urbán nos explica lo qu...,El eurodiputado Miguel Urbán nos explica lo qu...,actualidad,MLeon,twitter.com,https://www.meneame.net/story/eurodiputado-mig...,2023-12-30T19:48:00.000Z,2023-12-30T20:10:00.000Z,...,526,79200,Noche,Sábado,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
207,recz9kao4plaGgQlo,2024-03-21T21:39:10.000Z,"""La mujer es un enemigo, el bebé es un enemigo...","Eliyahu Yossian, analista israelí y veterano d...",actualidad,--557077--,twitter.com,https://www.meneame.net/story/mujer-enemigo-be...,2023-12-27T11:57:00.000Z,2023-12-27T14:55:00.000Z,...,430,640800,Mediodía,Miércoles,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000
208,reczk76lzg4WGFCTD,2024-03-21T21:39:02.000Z,Gypsy Rose Blanchard sale de la cárcel tras cu...,"Gypsy Rose Blanchard, de 32 años, se declaró c...",actualidad,Cesc_,bbc.com,https://www.meneame.net/story/gypsy-rose-blanc...,2023-12-28T13:30:00.000Z,2023-12-28T18:15:00.000Z,...,438,1026000,Tarde,Jueves,Diciembre,4to Trimestre,No encontrado,No encontrado,0.000000,0.000000


In [151]:
df_airtable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       210 non-null    object 
 1   createdTime              210 non-null    object 
 2   fields.Titular           210 non-null    object 
 3   fields.Entradilla        210 non-null    object 
 4   fields.Comunidad         210 non-null    object 
 5   fields.Usuario           210 non-null    object 
 6   fields.Medio             210 non-null    object 
 7   fields.URL               210 non-null    object 
 8   fields.Enviado           210 non-null    object 
 9   fields.Publicado         210 non-null    object 
 10  fields.Meneos            210 non-null    int64  
 11  fields.Clicks            210 non-null    int64  
 12  fields.Positivos         210 non-null    int64  
 13  fields.Anonimos          210 non-null    int64  
 14  fields.Negativos         2